# Data `staging` Step prior to data transformation

## Importing used library

In [1]:
import numpy as np
import pandas as pd 

## Data Loading for `staging` raw data

In [2]:
raw_daily = pd.read_csv('Raw_daily.csv', header=1)
raw_sales = pd.read_csv('Raw_sales.csv', index_col=0)
descriptions = pd.read_csv('Descriptions.csv')
raw_meta = pd.read_csv('Raw_meta.csv')

## Raw data `staging`

In [3]:
#raw_daily.info()

In [4]:
#raw_daily.head()

In [5]:
#raw_sales.info()

In [6]:
#raw_sales.head()

In [7]:
#descriptions.info()

In [8]:
#descriptions.head()

In [9]:
#raw_meta.info()

In [13]:
#raw_meta.columns = [col.replace('/', '_') for col in raw_meta.columns]

In [14]:
#raw_meta.info()

In [15]:
#raw_meta

### Observations in the structure of `raw_daily` DataFrame

- 1. Two columns representing index
- 2. uninformative column names

*Each one of the observations is addressed in the two cells below accordingly*

In [ ]:
raw_daily.drop('1', axis=1, inplace=True)

In [ ]:
raw_daily = raw_daily.rename(columns={
    'StnCode': 'Station code',
    'PD': 'Print run',
    'BC': 'Binding cost',
    'Unnamed: 5': 'Units sold',
    'RV': 'Returns',
    'Rev': 'Revenue', 
    'Tmp': 'Temperature (C)', 
    'Hmd': 'Humidity (%)',
    'VScr': 'Vendor score (0-10)', 
    'Typ': 'Product type'
})

In [ ]:
raw_daily.head()

In [ ]:
raw_daily.info()

### Observations in data

- 1. DateID is not in date format
-  2. Notes contains missing values
- 3. Poduct type contains wrongly encoded values
-  4. Returns contains negative values

*Each one of the observations is addressed in the cells below accordingly*

In [ ]:
raw_daily['DateID'] = pd.to_datetime(raw_daily['DateID'], format='mixed', errors='raise')

In [ ]:
raw_daily['Notes'] = raw_daily['Notes'].fillna('Nil')

In [ ]:
raw_daily['Product type'].unique()

In [ ]:
raw_daily['Product type'] = raw_daily['Product type'].str.replace('eâ€‘Book', 'e-Book')

In [ ]:
len(raw_daily[raw_daily['Returns'] < 0]['Returns'])

In [ ]:
def make_numeric_column_positive(dataframe):
    for col in dataframe.select_dtypes([int, float]): 
        dataframe[col] = dataframe[col].abs()
    return dataframe

raw_daily = make_numeric_column_positive(raw_daily)

In [ ]:
len(raw_daily[raw_daily['Returns'] < 0]['Returns'])

In [ ]:
raw_daily.head()

In [ ]:
raw_daily.info()

### Loading transformed data to flat file

In [ ]:
raw_daily.to_csv('raw_daily_processed.csv')

___

## Processing of Raw Sales

In [ ]:
raw_sales.head()

### Observation in the structure of `raw_sales` DataFrame
- 1. uninformative column names

*This is addressed in the cell below*

In [ ]:
raw_sales = raw_sales.rename(columns={
    'EdID': 'Edition ID',
    'Chnl': 'Channel code',
    'TQty': 'Quantity sold', 
    'UPrice': 'Unit price',
    'Curr': 'Currency',
    'Dscnt': 'Discount',
    'PD': 'Print run',
    'BC': 'Binding cost',
    'VScr': 'Vendor score (0-10)', 
    'Typ': 'Product type'
})

In [ ]:
raw_sales.head()

In [ ]:
raw_sales.info()

### Observations in data
- 1. Poduct type contains wrongly encoded values
- 2. DateStamp is not in date format
- 3. Discount has missing values

*Each one of the observations is addressed in the cells below accordingly*

In [ ]:
raw_sales['Product type'] = raw_sales['Product type'].str.replace('eâ€‘Book', 'e-Book')

In [ ]:
raw_sales['DateStamp'] = pd.to_datetime(raw_sales['DateStamp'], format='mixed', errors='raise')

In [ ]:
def fillna_with_mean(arr):
    arr = np.array(arr)
    total = np.nansum(arr)
    length = len(arr)
    mean = round((total / length), 3)
    np.nan_to_num(arr, copy=False, nan=mean)
    return arr

raw_sales['Discount'] = fillna_with_mean(raw_sales['Discount'])

In [ ]:
raw_sales.head()

In [ ]:
raw_sales.info()

### Loading transformed data to flat file

In [ ]:
raw_sales.to_csv('raw_sales_processed.csv')

# End of ETL

# Bonus

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Monthly Sales Trend

In [ ]:
daily_sales = raw_sales.groupby('DateStamp')['Quantity sold'].sum()

plt.figure(figsize=(14, 6)) 
plt.plot(daily_sales.index, daily_sales.values, linewidth=2, marker='o', markerfacecolor='r', mec='r') 
plt.title('Monthly Sales Trend', fontsize=16, fontweight='bold') 
plt.xlabel('Date')
plt.ylabel('Quantity Sold') 
plt.grid(True, alpha=0.3, linestyle='--') 
plt.xticks(rotation=45) 
plt.tight_layout() 
plt.show()

### Monthly Revenue Trend

In [ ]:
raw_sales['Revenue'] = raw_sales['Quantity sold'] * raw_sales['Unit price']
daily_revenue = raw_sales.groupby('DateStamp')['Revenue'].sum()

plt.figure(figsize=(14, 6))
plt.fill_between(daily_revenue.index, daily_revenue.values, alpha=0.3)
plt.plot(daily_revenue.index, daily_revenue.values, linewidth=2.5, color='blue')
plt.title('Monthly Revenue Trend', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Revenue')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Sales Trend by Channel

In [ ]:
channel_trends = raw_sales.groupby(['DateStamp', 'Channel code'])['Quantity sold'].sum().unstack()

plt.figure(figsize=(14, 7))

for channel in channel_trends.columns:
    plt.plot(channel_trends.index, channel_trends[channel], marker='o', label=channel, linewidth=2)

plt.title('Sales Trend by Channel', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Quantity Sold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Sales Trend by product type

In [ ]:
product_trends = raw_sales.groupby(['DateStamp', 'Product type'])['Quantity sold'].sum().unstack()

plt.figure(figsize=(14, 7))
plt.stackplot(
    product_trends.index, 
    *[product_trends[col] for col in product_trends.columns], 
    labels=product_trends.columns, alpha=0.8) 

plt.title('Sales Trend by Product Type', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Quantity Sold')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Sales Heatmap

In [ ]:
pivot_data = raw_sales.pivot_table(
    values='Quantity sold', 
    index='Channel code', 
    columns='DateStamp', 
    aggfunc='sum')

plt.figure(figsize=(16, 6))

sns.heatmap(pivot_data, cmap='YlGn', annot=False, cbar_kws={'label': 'Quantity Sold'})
plt.title('Sales Heatmap: Channel × Date', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Thank you